In [ ]:
using Plots
using DelimitedFiles
using LinearAlgebra

In [ ]:
moment_error_mc = readdlm("moment_error_mc.csv", ',')
moment_error_mc_eq = readdlm("moment_error_mc_eq.csv", ',')
sun_error_mc = readdlm("sun_error_mc.csv", ',')
sun_error_mc_eq = readdlm("sun_error_mc_eq.csv", ',')

In [ ]:
function plot_mc_sun(x_mc,N,thist,titletext)
    idx = 1:10:length(thist)
    plot(thist[idx]/3600,x_mc[idx,:],legend = false, color = :blue, alpha = 0.1, xlabel = "Time (hours)", ylabel = "Sun error (deg)", title = titletext, fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)
    average_sun_error = zeros(length(idx))
    for k = 1:length(idx)
        average_sun_error[k] = sum(x_mc[idx[k],:])/N
    end
    display(average_sun_error[end])
    plot!(thist[idx]/3600,average_sun_error, color = :red, linewidth = 2, label = "Average Sun error")
end
function plot_mc_momentum(x_mc,N,thist,titletext)
    idx = 1:10:length(thist)
    plot(thist[idx]/3600,x_mc[idx,:],legend = false, color = :blue, alpha = 0.1, xlabel = "Time (hours)", ylabel = "Normalized momentum error", title = titletext, fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)
    average_momentum_error = zeros(length(idx))
    for k = 1:length(idx)
        average_momentum_error[k] = sum(x_mc[idx[k],:])/N
    end
    display(average_momentum_error[end])
    plot!(thist[idx]/3600,average_momentum_error, color = :red,linewidth = 2, label = "Average Momentum error")
end

In [ ]:
Tfinal = 3*90*60;
Δt = 0.05
thist = 0:Δt:Tfinal

In [ ]:
plot_mc_sun(sun_error_mc,100,thist)
# savefig("sun_error_mc.pdf")

In [ ]:
sun_error_baseline = readdlm("sun_error_baseline.csv", ',')

In [ ]:
plot(thist/3600,sun_error_baseline,legend = false, color = :blue, xlabel = "Time (hours)", ylabel = "Sun error (deg)", title = "Sun pointing error", fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)

In [ ]:
sun_error_mc_baseline = readdlm("sun_error_mc_baseline.csv")

In [ ]:
plot_mc_sun(sun_error_mc_baseline,100,thist)
savefig("sun_error_mc_baseline.pdf")

In [ ]:
sun_error_mc_eq = readdlm("sun_error_mc_eq.csv", ',')

In [ ]:
plot_mc_sun(sun_error_mc_eq,100,thist)
savefig("sun_error_mc_eq.pdf")

In [ ]:
sun_error_mc_comb = readdlm("sun_error_mc_comb.csv")

In [ ]:
sun_error_mc_baseline_comb = readdlm("sun_error_mc_baseline_comb.csv")

In [ ]:
plot_mc_sun(sun_error_mc_baseline_comb,100,thist,"Baseline (SSO)")
savefig("sun_error_mc_baseline_comb_sso.pdf")

In [ ]:
plot_mc_sun(sun_error_mc_comb,100,thist,"Lyapunov Hybrid (SSO)")
savefig("sun_error_mc_comb_sso.pdf")

In [ ]:
function plot_mc_averages(x_mc,y_mc,N,thist)
    idx = 1:10:length(thist)
    average_x = zeros(length(idx))
    average_y = zeros(length(idx))
    for k = 1:length(idx)
        average_x[k] = sum(x_mc[idx[k],:])/N
        average_y[k] = sum(y_mc[idx[k],:])/N
    end
    plot(thist[idx]/3600,average_x, color = :red, linewidth = 2, label = "Lyapunov Hybrid")
    plot!(thist[idx]/3600,average_y, color = :blue, linewidth = 2, label = "Baseline", xlabel = "Time (hours)", ylabel = "Sun error (degrees)", fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)
end

In [ ]:
plot_mc_averages(sun_error_mc_comb,sun_error_mc_baseline_comb,100,thist)
savefig("sun_error_mc_comb_sso_vs_baseline.pdf")

In [ ]:
momentum_mc_comb = readdlm("momentum_mc_comb.csv")
momentum_mc_baseline_comb = readdlm("momentum_mc_baseline_comb.csv")

In [ ]:
plot_mc_momentum(momentum_mc_comb,100,thist,"Hybrid Lyapunov (SSO)")
savefig("momentum_mc_sso.pdf")

In [ ]:
plot_mc_momentum(momentum_mc_baseline_comb,100,thist)

In [ ]:
sun_error_baseline_comb_iss = readdlm("sun_error_mc_baseline_comb_iss.csv")
sun_error_mc_comb_iss = readdlm("sun_error_mc_comb_iss.csv")

In [ ]:
plot_mc_sun(sun_error_mc_comb_iss,100,thist,"Lyapunov Hybrid(LEO)")
savefig("sun_error_mc_comb_leo.pdf")

In [ ]:
plot_mc_sun(sun_error_baseline_comb_iss,100,thist,"Baseline (LEO)")
savefig("sun_error_mc_baseline_comb_leo.pdf")

In [ ]:
plot_mc_averages(sun_error_mc_comb_iss,sun_error_baseline_comb_iss,100,thist)
savefig("sun_error_mc_comb_leo_vs_baseline.pdf")

In [ ]:
x_sim_paper = readdlm("x_sim_paper.csv")

In [ ]:
function plot_which_lyapunov(x, thist)
    lyapunov = zeros(length(thist))
    h_hist = J_true*x[8:10,:]
    s_hist = x[5:7,:]
    h_target = 0.2*0.005
    spin_axis_target = major_axis
    for k = 1:length(thist)
        if max(norm(s_hist[:,k]-h_hist[:,k]/norm(h_hist[:,k])),0.15) > max(norm(spin_axis_target-h_hist[:,k]/h_target),0.26)
            lyapunov[k] = 1
        else
            lyapunov[k] = 2
        end
    end
    idx = 1:1:length(thist)
    h_hist = h_hist*1000
    plot(h_hist[1,idx],h_hist[2,idx],h_hist[3,idx],line_z = lyapunov[idx], c =:blues, legend=false,fontfamily = "Computer Modern", widen = true )
    #plot(thist,lyapunov) 
end

In [ ]:
J_true = readdlm("J_true.csv")

In [ ]:
#Inertia (from CAD)
J = [.0043 -.0003 0.0;
          -.0003 .0049 0.0;
            0.0   0.0 .0035]

umax = 50*.1*.1*0.1
major_axis = [-0.382683; 0.92388; 0.0]

In [ ]:
plot_which_lyapunov(x_sim_paper, thist)

In [ ]:
function plot_two_mcs_sun(x_mc1,x_mc2,N,thist)
    idx = 1:10:length(thist)
    layout = @layout [a b]
    p1 = plot(thist[idx]/3600,x_mc1[idx,:],legend = false, color = :blue, alpha = 0.1, xlabel = "Time (hours)", ylabel = "Sun error (deg)", title = "Lyapunov Hybrid Controller ", fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)
    average_sun_error1 = zeros(length(idx))
    for k = 1:length(idx)
        average_sun_error1[k] = sum(x_mc1[idx[k],:])/N
    end
    display(average_sun_error1[end])
    p1 = plot!(thist[idx]/3600,average_sun_error1, color = :red, linewidth = 2, label = "Average Sun error")
    p2 = plot(thist[idx]/3600,x_mc2[idx,:],legend = false, color = :blue, alpha = 0.1, xlabel = "Time (hours)", ylabel = "Sun error (deg)", title = "Baseline ", fontfamily = "Palatino", minorgrid = "true", xticks  = 0:0.5:4.5)
    average_sun_error2 = zeros(length(idx))
    for k = 1:length(idx)
        average_sun_error2[k] = sum(x_mc2[idx[k],:])/N
    end
    display(average_sun_error2[end])
    p2 = plot!(thist[idx]/3600,average_sun_error2, color = :red, linewidth = 2, label = "Average Sun error")
    plot(p1,p2,layout = layout, size = (1000, 400))
end

In [ ]:
plot_two_mcs_sun(sun_error_mc_comb_iss,sun_error_baseline_comb_iss,100,thist)